<a href="https://colab.research.google.com/github/Luana-lrb/LIPAI_Python/blob/main/onboarding/src/semana_10/pre_trained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Transfer Learning - ResNet18 pré treinada**

Com fins de estudo foi utilizado o dataset  Tiny ImageNet, devido a extensão do ImageNet original.

Importando bibliotecas necessárias:

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.models import resnet18, ResNet18_Weights
from tqdm import tqdm
import os
import zipfile
import shutil
import urllib.request

Device: usa GPU se disponível

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Treinando em:", device)
if device.type == "cuda":
    print("GPU:", torch.cuda.get_device_name(0))

Treinando em: cuda
GPU: Tesla T4



---
Função para carregar a Tiny ImageNet:




In [ ]:
def download_tiny_imagenet():
    tiny_imagenet_url = "http://cs231n.stanford.edu/tiny-imagenet-200.zip"
    data_dir = "./data"
    zip_path = os.path.join(data_dir, "tiny-imagenet-200.zip")
    extract_path = os.path.join(data_dir, "tiny-imagenet-200")

    os.makedirs(data_dir, exist_ok=True)
    if os.path.exists(extract_path):
        return extract_path
    if not os.path.exists(zip_path):
        # função pra mostrar o progresso
        def download_progress(block_num, block_size, total_size):
            downloaded = block_num * block_size
            percent = min(downloaded * 100.0 / total_size, 100)
            if block_num % 100 == 0:
                print(f"\rProgresso: {percent:.1f}% ({downloaded / (1024*1024):.1f} MB / {total_size / (1024*1024):.1f} MB)", end='')

        urllib.request.urlretrieve(tiny_imagenet_url, zip_path, download_progress)

    # extraindo os arquivos
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(data_dir)

    os.remove(zip_path)

    return extract_path


Função para organizar o diretório de validação do Tiny ImageNet

In [ ]:
def organize_tiny_imagenet_val(tiny_imagenet_path):
    val_dir = os.path.join(tiny_imagenet_path, 'val')
    val_annotations = os.path.join(val_dir, 'val_annotations.txt')
    images_dir = os.path.join(val_dir, 'images')

    if not os.path.exists(images_dir): return

    # ler anotações (mapeia imagem -> classe)
    img_to_class = {}
    with open(val_annotations, 'r') as f:
        for line in f:
            parts = line.strip().split('\t')
            img_name = parts[0]
            class_name = parts[1]
            img_to_class[img_name] = class_name

    # criar diretórios por classe
    for class_name in set(img_to_class.values()):
        class_dir = os.path.join(val_dir, class_name)
        os.makedirs(class_dir, exist_ok=True)

    # mover imagens para diretórios de classe
    for img_name, class_name in tqdm(img_to_class.items(), desc="Movendo imagens"):
        src = os.path.join(images_dir, img_name)
        dst = os.path.join(val_dir, class_name, img_name)
        if os.path.exists(src):
            shutil.move(src, dst)

    # remover diretório de imagens vazio e arquivo de anotações
    if os.path.exists(images_dir):
        os.rmdir(images_dir)
    if os.path.exists(val_annotations):
        os.remove(val_annotations)

Baixando e preparando Tiny ImageNet:

In [ ]:
tiny_imagenet_path = download_tiny_imagenet()

organize_tiny_imagenet_val(tiny_imagenet_path)



---



Preparando os Dataloaders



Usando modularização, para melhor visualização:

In [ ]:
def prepare_mnist_dataloader(batch_size=128):
    transform = transforms.Compose([
        transforms.Grayscale(num_output_channels=3),
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])  # Normalização ImageNet
    ])

    test_dataset = datasets.MNIST(
        root='./data',
        train=False,
        download=True,
        transform=transform
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=2,
        pin_memory=True
    )
    return test_loader

In [ ]:
def prepare_tiny_imagenet_dataloader(tiny_imagenet_path, batch_size=128):
    transform = transforms.Compose([
        transforms.Resize(224),  # 64x64 → 224x224
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])  # Normalização ImageNet
    ])

    val_path = os.path.join(tiny_imagenet_path, 'val')

    test_dataset = datasets.ImageFolder(
        root=val_path,
        transform=transform
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=2,
        pin_memory=True
    )
    return test_loader



---
Função para a avaliação genérica para servir tanto para o Tiny ImageNet quanto para o MNIST:


In [ ]:
def evaluate_model(model, dataloader, device, dataset_name, num_classes_dataset, class_mapping=None):
    model.eval()
    correct = 0
    total = 0
    correct_top5 = 0
    print(f"Avaliando o dataset: {dataset_name}")
    print(f"Classes no dataset: {num_classes_dataset}")
    print(f"Classes no modelo: {model.fc.out_features}")

    with torch.no_grad():
        for images, labels in tqdm(dataloader, desc=f"Testando"):
            images, labels = images.to(device), labels.to(device)

            # forward pass
            outputs = model(images)

            _, predicted = torch.max(outputs.data, 1)

            _, top5_predicted = torch.topk(outputs.data, 5, dim=1)
            # se houver mapeamento de classes, usar isso
            if class_mapping is not None:
                mapped_labels = torch.tensor([class_mapping.get(l.item(), -1) for l in labels]).to(device)
                valid_mask = mapped_labels != -1
                if valid_mask.sum() > 0:
                    correct += (predicted[valid_mask] == mapped_labels[valid_mask]).sum().item()

                    for i in range(len(labels)):
                        if valid_mask[i]:
                            if mapped_labels[i] in top5_predicted[i]:
                                correct_top5 += 1

                    total += valid_mask.sum().item()
            else:
                # Sem mapeamento, comparação direta
                correct += (predicted == labels).sum().item()

                for i in range(len(labels)):
                    if labels[i] in top5_predicted[i]:
                        correct_top5 += 1

                total += labels.size(0)

    accuracy = 100 * correct / total if total > 0 else 0
    accuracy_top5 = 100 * correct_top5 / total if total > 0 else 0

    print(f"   Total de imagens testadas: {total}")
    print(f"   Predições corretas: {correct}")
    print(f"   Predições incorretas: {total - correct}")
    print(f"   Acurácia: {accuracy:.2f}%")
    return accuracy




---
Execução principal


Carregando o modelo ResNet18 pré-treinado com o ImageNet

In [ ]:
weights = ResNet18_Weights.IMAGENET1K_V1
model = resnet18(weights=weights)
model = model.to(device)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
print(f"Classes de saída: {model.fc.out_features}")

Classes de saída: 1000


Dicionário onde vamos armazenar os resultados:

In [ ]:
resultados = {}

Primeiro testando o MNIST

In [ ]:
mnist_loader = prepare_mnist_dataloader(batch_size=256)
resultados['MNIST'] = evaluate_model(model, mnist_loader, device, "MNIST", num_classes_dataset=10)

Avaliando o dataset: MNIST
Classes no dataset: 10
Classes no modelo: 1000


Testando: 100%|██████████| 40/40 [00:19<00:00,  2.08it/s]

   Total de imagens testadas: 10000
   Predições corretas: 0
   Predições incorretas: 10000
   Acurácia: 0.00%


Agora testando o Tiny ImageNet

In [ ]:
tiny_loader = prepare_tiny_imagenet_dataloader(tiny_imagenet_path, batch_size=256)
resultados['Tiny ImageNet'] = evaluate_model(model, tiny_loader, device, "Tiny ImageNet", num_classes_dataset=200)

Avaliando o dataset: Tiny ImageNet
Classes no dataset: 200
Classes no modelo: 1000


Testando: 100%|██████████| 40/40 [00:24<00:00,  1.66it/s]

   Total de imagens testadas: 10000
   Predições corretas: 3
   Predições incorretas: 9997
   Acurácia: 0.03%


### **Analisando os resultados:**


> O modelo ResNet18 foi treinado para classificar exatamente 1000 classes
do ImageNet, cada uma com um índice específico (0-999).

> Como vimos na avaliação do MNIST, tem apenas 10 classes (dígitos 0-9) a acurácia é muito baixa: 0%. Porque os domínios são muito diferentes, dígitos manuscritos e objetos do mundo real.

> E no Tiny ImageNet tem 200 classes (subset do ImageNet), apresentando o problema de  MISMATCH de índices de classes e os 0.3% de acerto são conincidência quando, por acaso o índice no Tiny é igual ao índice no ImageNet.O modelo está semanticamente correrto, mas numericamente errado!

> Para obter um bom resultado seria necessário, para o MNIST, remover a última camada, adicionar nova camada final com 10 classes, fazer Fine-tuning, treinando apenas a nova camada e para Tiny ImageNet, mapear corretamente os índices.

> Se a avaliação fosse feita direto no dataset ImageNet completo, a acurácia deveria ser bem mais alta, devido a igualdade dos domínios de treinamento e avaliação.



